In [16]:
#Importing standard libraries
import sys
import time
from pathlib import Path
from typing import Optional, cast 

#Importing external modules
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh, LinearOperator

#Import local modules
module_dir = r"C:\Users\sanvi\Documents\Github\Efficient-Simulation-of-the-Schrodinger-Equation"
sys.path.append(module_dir)
import utils.numerical as numerical
import utils.standard_ho as sho
import utils.visualisation as vis

#Import  QuEvolutio modules
from quevolutio.core.aliases import (  # isort: skip
    RVector,
    GVector,
    CTensors,
    CSRMatrix,
)
from quevolutio.core.domain import QuantumConstants, QuantumHilbertSpace, TimeGrid
from quevolutio.core.tdse import TDSE, Controls, Hamiltonian
from quevolutio.propagators.semi_global import ApproximationBasis, SemiGlobal

In [17]:
class DHOConstants(QuantumConstants):
    
    hbar: float = 1.0
    mass: float = 1.0
    v0: float = 5.0
    kl: float = 1.0
    alpha: float = 10.0
    q: float = 0.0
    band: int = 0
    N: int = 10
    unit_cell: float = np.pi / kl
    num_pts: int = 200
    dims: int = 2*N +1 
constants = DHOConstants()

In [18]:
class OpticalLatticeHamiltonian(Hamiltonian):
    time_dependent: bool = False
    ke_time_dependent: bool = False
    pe_time_dependent: bool = False

    def __init__(
        self, domain: QuantumHilbertSpace, eigenvalue_min: float, eigenvalue_max: float
    ) -> None:
        # Assign attributes.
        self.domain: QuantumHilbertSpace = domain
        self.eigenvalue_min: float = eigenvalue_min
        self.eigenvalue_max: float = eigenvalue_max

        # For static type checking (not runtime required).
        self.domain.constants = cast(DHOConstants, self.domain.constants)

        # Pre-compute the kinetic energy diagonal.
        self._ke_diagonal: RVector = -((domain.momentum_axes[0]+DHOConstants.q) ** 2) / (
            2.0 * self.domain.constants.mass
        )
        self._ke_diagonal /= constants.v0 * constants.alpha
        # Pre-compute the lattice potential energy diagonal.
        self._pe_diagonal: RVector = (
            -0.5
            * self.domain.constants.v0
            * np.cos(2 * self.domain.constants.kl * self.domain.position_axes[0])
        )
        self._pe_diagonal /= constants.v0 * constants.alpha

    def __call__(self, state: GVector, controls: Optional[Controls] = None) -> GVector:
            """
            Calculates the action of the Hamiltonian on a state. If the Hamiltonian
            has explicit time dependence, a set of controls should be passed.

            Parameters
            ----------
            state : GVector
                The state being acted on. This should have shape
                (*domain.num_points).
            controls : Optional[Controls]
                The controls which determine the structure of the Hamiltonian. This
                should be passed if the Hamiltonian has explicit time dependence.

            Returns
            -------
            GVector
                The result of acting the Hamiltonian on the state.
            """

            return self.ke_action(state) + self.pe_action(state)
    def ke_action(self, state: GVector, controls: Optional[Controls] = None) -> GVector:
        """
        Calculates the action of the kinetic energy operator on a state. If the
        kinetic energy operator has explicit time dependence, a set of controls
        should be passed.

        Parameters
        ----------
        state : GVector
            The state being acted on. This should have shape
            (*domain.num_points).
        controls : Optional[Controls]
            The controls which determine the structure of the Hamiltonian. This
            should be passed if the kinetic energy operator has explicit time
            dependence.

        Returns
        -------
        GVector
            The result of acting the kinetic energy operator on the state.
        """
       
        return self.domain.position_space(
            self._ke_diagonal * self.domain.momentum_space(state)
        )

    def pe_action(self, state: GVector, controls: Optional[Controls] = None) -> GVector:
        """
        Calculates the action of the potential energy operator on a state. If
        the potential energy operator has explicit time dependence, a set of
        controls should be passed.

        Parameters
        ----------
        state : GVector
            The state being acted on. This should have shape
            (*domain.num_points).
        controls : Optional[Controls]
            The controls which determine the structure of the Hamiltonian. This
            should be passed if the potential energy operator has explicit time
            dependence.

        Returns
        -------
        GVector
            The result of acting the potential energy operator on the state.
        """

        return (
            self._pe_diagonal * state
        )
    
def controls_fn(time: float) -> Controls: 
     """
     Evaluates the controls which determine the structure of the time-dependent
     Schrödinger equation (TDSE) for a 1D driven harmonic oscillator system. In
    this case, this is just the time.

     Parameters
     ----------
    time : float
         The time at which to evaluate the controls.

     Returns
    -------
     Controls
         The controls for the TDSE at the given time.
     """

     return time 



In [ ]:
#using the previous H_builder function to create some test eigenvalues and eigenvectors
domain: QuantumHilbertSpace = QuantumHilbertSpace(
        num_dimensions=1,
        num_points=np.array([DHOConstants.num_pts]),
        position_bounds=np.array([[-10.0, 10.0]]),
        constants=constants,
    )

H = OpticalLatticeHamiltonian(domain, 0, 1.0) #guessed eigenvalue range


#l = np.arange(-DHOConstants.N, DHOConstants.N+1)
#c = eigenvectors[:, DHOConstants.band]


constants: DHOConstants = DHOConstants()

#setting up position grid
x_grid = domain.position_axes[0]       # 1D position grid
N = len(x_grid)
dx = x_grid[1] - x_grid[0]                  # grid spacing
domain.momentum_axes = [2*np.pi*np.fft.fftfreq(N, d=dx)] # setting up momentum grid

def matvec(v):
    psi = v.reshape((N,))
    return np.asarray(H(psi, None), dtype=np.complex128)

A = LinearOperator((N, N), matvec=matvec, dtype=np.complex128)

k = 6
evals, evecs = eigsh(A, k=k, which="SA")

idx = np.argsort(evals)
evals = evals[idx]
evecs = evecs[:, idx]




def normalize(state: GVector) -> GVector:
    prob = np.abs(state)**2
    #dx = DHOConstants.unit_cell / (DHOConstants.num_pts-1)
    norm = np.trapezoid(prob, dx=dx)
    return state / np.sqrt(norm)

u0 = evecs[:, 0]
psi0 = np.exp(1j * DHOConstants.q * x_grid) * u0


print("lowest eigenvalues:", evals)

emin = float(evals[0])
emax = float(evals[-1])


#check normalisation
psi0 = normalize(psi0)


lowest eigenvalues: [-39.08228832 -38.30963879 -38.30033241 -37.53837315 -37.52856112
 -36.77273608]


In [20]:
hamiltonian: OpticalLatticeHamiltonian = OpticalLatticeHamiltonian(domain, emin, emax)
tdse: TDSE = TDSE(domain, hamiltonian)
time_domain: TimeGrid = TimeGrid(time_min=0.0, time_max=10.0, num_points=10001)

propagator = SemiGlobal(
tdse,
time_domain,
order_m=10,
order_k=10,
tolerance=1e-5,
approximation=ApproximationBasis.NEWTONIAN,
)
    # Propagate the state.
print("Propagation Start")
start_time: float = time.time()
states: CTensors = propagator.propagate(
    psi0, diagnostics=True
)
final_time: float = time.time()
print("Propagation Done")
   # Calculate the norms of the states.
norms: RVector = numerical.states_norms(states, domain)

   # Calculate the position expectation values of the states.
#states_expectation: RVector = cast(
    #RVector,
  #  np.trapezoid(
      #  ((np.abs(states) ** 2) * domain.position_axes[0]),
      #  dx=domain.position_deltas[0],
        #axis=1,
     # ),
 #  )

print ( states, norms)

Propagation Start
Time Index: 0 	 Iteration: 1
Convergence: 3.90798e-02
Time Index: 0 	 Iteration: 2
Convergence: 0.00000e+00
Time Index: 1 	 Iteration: 1
Convergence: 1.07851e-13
Time Index: 2 	 Iteration: 1
Convergence: 1.07816e-13
Time Index: 3 	 Iteration: 1
Convergence: 1.07945e-13
Time Index: 4 	 Iteration: 1
Convergence: 1.07919e-13
Time Index: 5 	 Iteration: 1
Convergence: 1.07855e-13
Time Index: 6 	 Iteration: 1
Convergence: 1.07905e-13
Time Index: 7 	 Iteration: 1
Convergence: 1.07911e-13
Time Index: 8 	 Iteration: 1
Convergence: 1.07892e-13
Time Index: 9 	 Iteration: 1
Convergence: 1.07840e-13
Time Index: 10 	 Iteration: 1
Convergence: 1.07801e-13
Time Index: 11 	 Iteration: 1
Convergence: 1.07802e-13
Time Index: 12 	 Iteration: 1
Convergence: 1.07813e-13
Time Index: 13 	 Iteration: 1
Convergence: 1.07848e-13
Time Index: 14 	 Iteration: 1
Convergence: 1.07831e-13
Time Index: 15 	 Iteration: 1
Convergence: 1.07840e-13
Time Index: 16 	 Iteration: 1
Convergence: 1.07825e-13
Tim

In [21]:
filename: str = "Optical_Lattice_SemiGlobal_1D_2"

# Create directories for saving data if they do not exist
for folder in (Path("data"), Path("figures"), Path("anims")):
    folder.mkdir(parents=True, exist_ok=True)

# Save the propagated states
np.save(f"data/{filename}.npy", states)

# Plot the propagated states
vis.plot_state_1D(
    states[0],
    domain,
    r"$\text{State} \; (T = 0.00)$",
    f"figures/{filename}_start.png",
)

vis.plot_state_1D(
    states[-1],
    domain,
    rf"$\text{{State}} \; (T = {time_domain.time_axis[-1]:.2f})$",
    f"figures/{filename}_final.png",
)


findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
findfont: Font family 'Lato' not found.
